In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
model = ChatOpenAI(temperature=0.1)

In [ ]:
haiku_template = ChatPromptTemplate.from_template(
    """You are a poet who specializes in writing haikus about programming languages.
    A haiku is a three-line poem where the first line has 5 syllables, the second line has 7 syllables, 
    and the third line has 5 syllables.
    
    Write a creative and insightful haiku about the programming language: {language}.
    
    Only return the haiku itself, nothing else."""
)

In [ ]:
# Create the haiku generation chain
haiku_chain = haiku_template | model | StrOutputParser()

In [ ]:
# Define the prompt template for explaining haikus
explanation_template = ChatPromptTemplate.from_template(
    """You are an expert in literary analysis, specializing in haikus.
    
    Analyze and explain the following haiku about {language} programming language:
    
    {haiku}
    
    Provide a thoughtful explanation of the imagery, metaphors, and how it relates to the {language} programming language.
    Your explanation should be informative but concise."""
)

In [ ]:
# Create the explanation chain
explanation_chain = explanation_template | model | StrOutputParser()

In [ ]:
# Define the combined chain
def combine_haiku_and_explanation(inputs):
    return {
        "haiku": inputs["haiku"],
        "explanation": inputs["explanation"],
        "language": inputs["language"],
    }


# Create the final chain using LCEL
final_chain = (
    {"language": RunnablePassthrough()}
    | {"haiku": haiku_chain, "language": RunnablePassthrough()}
    | {
        "haiku": lambda x: x["haiku"],
        "explanation": explanation_chain,
        "language": lambda x: x["language"],
    }
    | combine_haiku_and_explanation
)

In [ ]:
# Test the chain with a programming language
result = final_chain.invoke("Python")

print(f"Programming Language: {result['language']}\n")
print(f"Haiku:\n{result['haiku']}\n")
print(f"Explanation:\n{result['explanation']}")